In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
# Import các thư viện
from typing import List
import numpy as np
import torch
import evaluate
from sklearn.model_selection import train_test_split
import nltk
from transformers import AutoTokenizer, AutoModelForTokenClassification, Trainer, TrainingArguments
from torch.utils.data import Dataset

In [ ]:
# Penn Treebank: Đây là bộ dữ liệu gồm các câu đã được gắn nhãn từ loại (POS tags)
# Download dataset từ package nltk
nltk.download("treebank")

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

In [ ]:
# Trả về danh sách các câu đã được gắn nhãn, sử dụng phương thức tagged_sents()
tagged_sentences = nltk.corpus.treebank.tagged_sents()
# Mỗi phần tử là một danh sách các cặp (word, tag)
print("Số lượng các câu đã được đánh nhãn: ", len(tagged_sentences))

Số lượng các câu đã được đánh nhãn:  3914


In [ ]:
tagged_sentences[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [ ]:
# Tách câu và nhãn của từng loại từ ra và lưu vào hai danh sách tách biệt
sentences, sentence_tags = [], []
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append([word.lower() for word in sentence])
    # sentence_tags.append(list(tags))
    sentence_tags.append([tag for tag in tags])

In [ ]:
# Tạo một bộ chứa tất cả các loại nhãn (label) trong dataset Penn Treebank
unique_tags = set(tag for sentence_tags in sentence_tags for tag in sentence_tags)
# Tạo một bộ index cho tag
label_to_id = {tag: idx for idx, tag in enumerate(unique_tags)}
# Tạo một bộ chuyển index về tag (phục vụ cho bước dự đoán - model inference)
id_to_label = {idx: tag for tag, idx in label_to_id.items()}
# Tạo biến lưu lại số index của các tag
num_labels = len(label_to_id)

In [ ]:
# Tách dữ liệu thành tập train, test, và valid
# X: sentence
# y: sentence tags
train_sentences, test_sentences, train_tags, test_tags = train_test_split(sentences, sentence_tags,  test_size=0.3, random_state=42)
valid_sentences, test_sentences, valid_tags, test_tags = train_test_split(test_sentences, test_tags, test_size=0.5, random_state=42)

In [ ]:
# Load pretrained model QCRI/bert-base-multilingual-cased-pos-english và finetune trên tập dữ liệu Penn Treebank để phục vụ cho bài toán POS tagging
model_name = "QCRI/bert-base-multilingual-cased-pos-english"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
# Định nghĩa class quản lý Dataset cho môi trường Fine-tune
MAX_LEN = 128
class POSTagging_Dataset(Dataset):
    def __init__(self, sentences: List[List[str]], tags: List[List[str]], tokenizer, label_to_id, max_len=MAX_LEN):
        super().__init__()
        self.sentences = sentences
        self.tags = tags
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.label_to_id = label_to_id

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        # Tokenize sentence
        encoding = self.tokenizer(
            self.sentences[idx],
            is_split_into_words=True, # Báo cho tokenizer rằng là input của mình là danh sách từ, không phải là một câu
            return_tensors="pt", # Trả về một Pytorch tensor
            padding="max_length", # Thêm (pad) để đảm bảo các câu có cùng độ dài
            truncation=True, # Cắt bớt chuỗi nếu quá dài
            max_length=self.max_len,
            return_offsets_mapping=False # Không cần trả về index của ký tự
        )

        # Sắp xếp labels với các từ trong câu
        word_ids = self.tokenizer(self.sentences[idx], is_split_into_words=True).word_ids()
        labels = [-100] * self.max_len
        for i, word_id in enumerate(word_ids):
            if word_id is not None and i < self.max_len:
                if labels[i] == -100:
                    labels[i] = self.label_to_id[self.tags[idx][word_id]]

        return {
            "input_ids": encoding['input_ids'].squeeze(),
            "attention_mask": encoding["attention_mask"].squeeze(),
            "labels": torch.as_tensor(labels)
        }

In [ ]:
# Tạo các data loader
train_dataloader = POSTagging_Dataset(train_sentences, train_tags, tokenizer, label_to_id)
val_dataloader = POSTagging_Dataset(valid_sentences, valid_tags, tokenizer, label_to_id)
test_dataloader = POSTagging_Dataset(test_sentences, test_tags, tokenizer, label_to_id)

In [ ]:
model = AutoModelForTokenClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    id2label=id_to_label,
    label2id=label_to_id
)

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
# Định nghĩa metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred # dự đoán cho tất cả token, kể cả [PAD], predictions là xác suất thô cho từng lớp
    predictions = np.argmax(predictions, axis=-1) # Lấy ra lớp có xác suất cao nhất

    # Chỉ lấy các token không bị ignore (-100)
    true_labels = []
    pred_labels = []
    for pred, label in zip(predictions, labels):
        for p, l in zip(pred, label):
            if l != -100:  # Lọc bỏ các token bị ignore (-100) như [PAD], [CLS], [SEP]
            # [CLS]: luôn đứng đầu câu
            # [SEP]: phân tách 2 đoạn văn bản trong cùng một câu văn bản
            # [PAD]: thêm vào để các câu có cùng độ dài
                true_labels.append(l)
                pred_labels.append(p)

    # Tính accuracy
    accuracy = np.mean(np.array(true_labels) == np.array(pred_labels))

    return {
        "accuracy": accuracy
    }

In [ ]:
# Định nghĩa training arguments
training_args = TrainingArguments(
    output_dir="./pos_tagging_output",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    logging_steps=100,
    seed=42
)

In [ ]:
# Khởi tạo trainer
trainer = Trainer(
    model,
    args=training_args,
    train_dataset=train_dataloader,
    eval_dataset=val_dataloader,
    compute_metrics=compute_metrics
)

# Train model: quá trình finetune được bắt đầu tại đây
trainer.train()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: cybersoft-codingcamp (cybersoft-codingcamp-cybersoft-academy-o-t-o-chuy-n-gia-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.722900,0.110810,0.966589
2,0.087700,0.096118,0.974231
3,0.055300,0.094644,0.975769


TrainOutput(global_step=516, training_loss=0.40368047784003175, metrics={'train_runtime': 633.4087, 'train_samples_per_second': 12.973, 'train_steps_per_second': 0.815, 'total_flos': 536982389881344.0, 'train_loss': 0.40368047784003175, 'epoch': 3.0})

In [ ]:
test_results = trainer.evaluate(test_dataloader)
print(f"Kết quả test: {test_results}")

Kết quả test: {'eval_loss': 0.09389744699001312, 'eval_accuracy': 0.9736047904191617, 'eval_runtime': 5.5915, 'eval_samples_per_second': 105.16, 'eval_steps_per_second': 6.617, 'epoch': 3.0}


In [ ]:
# Hàm dự đoán cho một câu
def predict_pos_tags(sentence: str):
    # Tokenize câu
    inputs = tokenizer(
        sentence.split(),
        is_split_into_words=True,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=MAX_LEN
    )

    # Chuyển sang device phù hợp
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Dự đoán
    with torch.no_grad():
        outputs = model(**inputs)

    predictions = torch.argmax(outputs.logits, dim=-1).cpu().numpy()[0]
    word_ids = tokenizer(sentence.split(), is_split_into_words=True).word_ids()

    # Lấy tag cho subword đầu tiên của mỗi từ
    pred_tags = []
    current_word_id = None
    for i, word_id in enumerate(word_ids):
        if word_id is not None and word_id != current_word_id:
            pred_tags.append(id_to_label[predictions[i]])
            current_word_id = word_id

    return list(zip(sentence.split(), pred_tags))

In [ ]:
test_sentence = "We are exploring the topic of deep learning"
print(predict_pos_tags(test_sentence))

[('We', 'PRP'), ('are', 'VBP'), ('exploring', 'VBG'), ('the', 'DT'), ('topic', 'NN'), ('of', 'IN'), ('deep', 'JJ'), ('learning', 'NN')]
